# Conversation Generator

In [1]:
!pip install -q openai

import os
import openai
from time import time, sleep
import textwrap
import re
import requests
import threading
from IPython.display import clear_output, display
import ipywidgets as widgets
from datetime import datetime
from random import randint
from datetime import datetime as dt
from enum import Enum


## File Management

In [2]:
def read_from_file(filepath):
    with open(filepath, 'r', encoding='utf-8') as infile:
        return infile.read()

def write_to_file(filepath, content):
    with open(filepath, 'w', encoding='utf-8') as outfile:
        outfile.write(content)

def append_to_file(filepath, content):
    with open(filepath, 'a', encoding='utf-8') as outfile:
        outfile.write(content)
        
def ensure_folder_exists(folder: str) -> None:
    if not os.path.exists(folder):
        os.mkdir(folder)

def ensure_file_exists(folder: str, filename: str) -> None:
    ensure_folder_exists(folder)
    if not os.path.exists(folder + '/' + filename):
        write_to_file(folder + '/' + filename, '')

def get_all_text_files(folder):
    files = os.listdir(folder)
    files = [f for f in files if '.txt' in f]
    write_to_file('full draft/book.txt', '')
    for file in files:
        content = read_from_file('{0}/{1}'.format(folder, file))
        append_to_file('full draft/book.txt', content + '\n')
    print('done')
    
def get_filename(folder: str, msg: str) -> 'str':
    while True:
        sleep(.3)
        filename = input(msg)
        if os.path.isfile('{0}/{1}'.format(folder, filename)):
            print('File loaded successfully')
            return '{0}/{1}'.format(folder, filename)
        clear_output()
        print('Invalid filename. Try again.')

## Prompt Management

In [3]:
def load_openai_api_key() -> None:
    '''This loads the openai API key from openaiapikey.txt'''
    ensure_folder_exists('api_key')
    files = os.listdir('api_key')
    files = [f for f in files if '.txt' in f]
    if files == []:
        get_api_key()
        write_to_file('api_key/openai.txt', openai.api_key)
        return
    openai.api_key = read_from_file('api_key/' + files[0])
    return

def get_api_key() -> None:
    while True:
        print('You will need an openai api key. They can be found here:')
        print('https://platform.openai.com/account/api-keys')
        sleep(.3)
        openai.api_key = input('Enter your openai api key:')
        clear_output()
        if validate_api_key():
            return
        print('Invalid API key.\n')

def validate_api_key() -> bool:
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[{'role':'user', 'content':'Is this api key working?'}])
    except:
        return False
    return True

def send_prompt(messages: str) -> str:
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[{'role':'user', 'content':messages}])
    except:
        print('ChatGPT rate limit reached. Resuming in one minute...')
        sleep(61)
        return send_prompt(messages)
    text = response['choices'][0]['message']['content']
    return text.strip()

class Requester_Thread(threading.Thread):
    def __init__(self, prompt: str):
        threading.Thread.__init__(self)
        self.prompt = prompt
        
    def run(self):
        self.value = send_prompt(self.prompt)

## Persona Generator

In [4]:
def get_random_location():
    prompt = '''Ignore all previous instructions. State a random place 
    in the world where people live. Only return the place name. No descriptions,
    headers, punctuation, or any other text.'''
    return send_prompt(prompt)

def get_name_list(location: str):
    prompt = f'''Ignore all previous instructions. Return a list of 50 random 
    that sound like they belong to people from {location}. Return the list of
    first and last names in a new line separated format with no numbering or 
    any other kind of punctuation. The first 25 names must be female names while
    the last 25 must be male names. Do not return any headers, descriptions, or
    other text.'''
    return send_prompt(prompt)

class Gender(Enum):
    FEMALE = 1
    MALE = 2

class Persona():
    def __init__(self, location: str, names: list, name=None, gender=None):
        self.location = location
        self.set_gender(gender)
        self.set_name(names, name)
        self.set_personality()
        self.age = randint(18,91)
        
    def set_gender(self, gender=None):
        if gender:
            self.gender = gender
        elif randint(0,1) > 0:
            self.gender = Gender.MALE
        else:
            self.gender = Gender.FEMALE
    
    def set_name(self, names, name=None):
        if name:
            self.name = name
        elif self.gender == Gender.MALE:
            self.name = names[randint(25,50)]
        else:
            self.name = names[randint(0,25)]
        self.surname = self.name.split(' ')[1]
        
    def random_severity():
        severity = ['extremely', 'very', 'moderately', 'not very', 'not']
        return severity[randint(0,5)]
    
    def set_personality():
        bigfive = {}
        bigfive['openness'] = f'Openness to experience: {random_severity()} open to new experiences.'
        bigfive['conscience'] = f'Conscientiousness: {random_severity()} organized and/or efficient.'
        bigfive['extraversion'] = f'Extraversion: {random_severity()} outgoing and/or socialable.'
        bigfive['agreeableness'] = f'Agreeableness: {random_severity()} agreeable and/or friendly.'
        bigfive['neuroticism'] = f'Neuroticism: {random_severity()} nervous and/or sensitive.'
        self.bigfive = bigfive
        

## Main Function

In [6]:
def main():
    load_openai_api_key()
    topic = get_book_topic()
    if topic is None:
        return
    chapters = get_chapter_titles(topic)
    title = get_book_title(topic, chapters)
    chapter_outline = get_main_ideas(topic, chapters)
    generate_draft(topic, chapter_outline, title)

#main()

In [7]:
#load_openai_api_key()
#location = get_random_location()
#print(get_name_list(location))

In [9]:
def build_gui():
    msg_list = []
    
    def handle_send(sender):
        if chat_field.value != '':
            msg_list.append(chat_field.value)
            with messages:
                clear_output()
                print('\n\n'.join(msg_list))
                chat_field.value = ''
    
    
    
    side_layout = widgets.Layout(width='200px', height='300px')
    main_layout = widgets.Layout(width='400px', height='300px', overflow_y='auto')
    chat_layout = widgets.Layout(width='400px')
    butt_layout = widgets.Layout(width='200px')
    
    controls = widgets.Output(layout=side_layout)
    messages = widgets.Output(layout=main_layout)
    chatters = widgets.Output(layout=side_layout)
    emptybox = widgets.Output(layout=butt_layout)
    
    chat_field = widgets.Text(continuous_update=False, layout=main_layout)
    chat_field.observe(handle_send, 'value')
    send_btn = widgets.Button(description='Send')
    send_btn.on_click(handle_send)    
    
    messagebar = widgets.HBox([controls, messages, chatters])
    chatbar = widgets.HBox([emptybox, chat_field, send_btn])
    gui = widgets.VBox([messagebar, chatbar])
    display(gui)
    
build_gui()